<a href="https://colab.research.google.com/github/nakyeong-kim/python_advanced/blob/main/4_context_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Context Manager, Context Manager Annotation

### Contextlib, \_\_enter\_\_, \_\_exit\_\_, exception, @contextlib.contextmanager <br><br>    


* Context Manager : 원하는 타이밍에 정확하게 리소스를 할당, 제공, 반환하는 역할<br>
  - 대표적인 것이 with문
* \_\_enter\_\_(self) : with문에 진입하는 시점에 자동으로 호출되는 메소드
* \_\_exit\_\_(self, type, value, traceback) : with문을 빠져나오기 직전에 호출되는 메소드
  - type, value, traceback는 with문을 빠져나오기 전에 예외가 발생했을 때의 정보를 나타냄
* Conetxtlib 데코레이터 사용 → 직관적인 코드, 예외처리 용이
<br><br><br>

In [19]:
# Ex1

file = open('./testfile1.txt', 'w')
try:
  file.write('Context Manager Test1\nContextlib Test1.')
finally:
  file.close()    # 자원을 할당 받았으므로 다시 돌려줘야 함

In [20]:
# Ex2

with open('./testfile2.txt', 'w') as f:
  f.write('Context Manager Test2\nContextlib Test2.')

In [21]:
# Ex3
# Use Class - Context Manager with exception hadling
# Class를 선언하고 __enter__와 __exit__ 메소드를 선언하면 Context Manager가 됨

class MyFileWriter():
  def __init__(self, file_name, method):
    print('MyFileWriter started : __init__')
    self.file_obj = open(file_name, method)

  def __enter__(self):
    print('MyFileWriter started : __enter__')
    return self.file_obj

  def __exit__(self, exc_type, exc_value, exc_traceback):
    print('MyFileWriter started : __exit__')
    if exc_type:
      print('Logging exception {}'.format((exc_type, exc_value, exc_traceback)))
    self.file_obj.close()

with MyFileWriter('./testfile3.txt', 'w') as f:
  f.write('Context Manager Test3\nContextlib Test3.')

MyFileWriter started : __init__
MyFileWriter started : __enter__
MyFileWriter started : __exit__


In [24]:
# Ex4
# Use Class
# Contextlib - Measure execution(타이머) 제작

import time

class ExcuteTimer(object):    # ()도 무관하지만, 엄격한 선언을 위해 object 명시 (모든 클래스는 오브젝트를 상속받는다)
  def __init__(self, msg):
    self._msg = msg

  def __enter__(self):
    self._start = time.monotonic()
    return self._start

  def __exit__(self, exc_type, exc_value, exc_traceback):
    if exc_type:
      print('Logging exception {}'.format((exc_type, exc_value, exc_traceback)))
    else:
      print('{} : {} s'.format(self._msg, time.monotonic() - self._start))
    return True   # 엄격한 코딩에서 명시 : with문을 잘 빠져나왔음을 뜻함

with ExcuteTimer('Start job!') as v:
  print('Received start monotonic1 : {}'.format(v))
  # Excute job
  for i in range(10000000):
    pass
  raise Exception('Raise! Exception!!')   # 강제로 예외 발생

Received start monotonic1 : 3777.308676827
Logging exception (<class 'Exception'>, Exception('Raise! Exception!!'), <traceback object at 0x7ac4d6f36b40>)


In [25]:
# Ex5
# Use decorator - 훨씬 간단
# 예외 처리를 엄격하게 할 때에는 class 방식을 사용하는 것이 좋으나, 함수 형태를 쓰는 오픈 소스들이 많이 보임

import contextlib
import time

@contextlib.contextmanager
def my_file_writer(file_name, method):
  f = open(file_name, method)
  yield f     # __enter__ 구문
  f.close()   # __exit__ 구문

with my_file_writer('./testfile4.txt', 'w') as f:
  f.write('Context Manager Test4\nContextlib Test4.')

In [35]:
# Ex6
# Use decorator

import contextlib
import time

@contextlib.contextmanager
def ExcuteTimerDc(msg):
  start = time.monotonic()
  try :   # __enter__
    yield start
  except BaseException as e:
    print('Logging exception: {}: {}'.format(msg, e))
    raise
  else:   # __exit__
    print('{}: {}s'.format(msg, time.monotonic() - start))

with ExcuteTimerDc('Start job!') as v:
  print('Received start monotonic2 : {}'.format(v))
  # Excute job
  for i in range(10000000):
    pass
  # raise ValueError('occurred.')   # 강제로 예외 발생

Received start monotonic2 : 4404.454526497
Start job!: 0.23687274799976876s
